In [ ]:
#引入库
import os
import pandas as pd
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

# export data

**导出Normal和年龄相关性的位点数据**

In [ ]:
meta = pd.read_csv('../../train_data/meta.csv')
merge = pd.read_csv('../../train_data/merge_Blood_withchr.csv',index_col=0)
merge.index = [str(i) for i in merge.index.tolist()]
df = pd.read_csv('../../train_data/bootstrap_Blood_0.2.csv',index_col=0)
merge = merge.loc[df.columns.tolist()[:-1]]

# 将merge data匹配age信息
merge_data = pd.merge(merge.T,meta[["SRA","age","disease state"]],left_index=True,right_on=["SRA"])
merge_data = merge_data.set_index("SRA")

merge_data2 = merge_data.loc[merge_data['disease state'] == 'Normal',:]

merge_check = merge_data2.iloc[:,0:merge.shape[0]].isnull()

# 计算单个cpgsite与age的相关性
merge['corr'] = [np.corrcoef(merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),i].tolist(), merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),-2].tolist())[0,1] for i in range(merge_data2.shape[1]-2)]

# 将corr列从高到低排序
merge1 = merge.sort_values(by='corr',ascending=False)
# 删除corr为NA的行
merge2 = merge1.loc[~merge1['corr'].isna(),:]
# 保存数据
nums = [500,1000,2000,3000]
for num in nums:
    merge2.iloc[0:num,0:3].to_csv(f'normal_positive_{num}.csv')
    merge2.iloc[-num:,0:3].to_csv(f'normal_negative_{num}.csv')

**导出Disease和年龄相关性的位点数据**

In [ ]:
meta = pd.read_csv('../../train_data/meta.csv')
merge = pd.read_csv('../../train_data/merge_Blood_withchr.csv',index_col=0)
merge.index = [str(i) for i in merge.index.tolist()]
df = pd.read_csv('../../train_data/bootstrap_Blood_0.2.csv',index_col=0)
merge = merge.loc[df.columns.tolist()[:-1]]

# 将merge data匹配age信息
merge_data = pd.merge(merge.T,meta[["SRA","age","disease state"]],left_index=True,right_on=["SRA"])
merge_data = merge_data.set_index("SRA")

merge_data2 = merge_data.loc[merge_data['disease state'] != 'Normal',:]

merge_check = merge_data2.iloc[:,0:merge.shape[0]].isnull()

# 计算单个cpgsite与age的相关性
merge['corr'] = [np.corrcoef(merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),i].tolist(), merge_data2.iloc[(~merge_check.iloc[:,i]).tolist(),-2].tolist())[0,1] for i in range(merge_data2.shape[1]-2)]

# 将corr列从高到低排序
merge1 = merge.sort_values(by='corr',ascending=False)
# 删除corr为NA的行
merge2 = merge1.loc[~merge1['corr'].isna(),:]
# 保存数据
nums = [500,1000,2000,3000]
for num in nums:
    merge2.iloc[0:num,0:3].to_csv(f'disease_positive_{num}.csv')
    merge2.iloc[-num:,0:3].to_csv(f'disease_negative_{num}.csv')

# GREAT analysis

将上述得到的数据输入到[GREAT](http://great.stanford.edu/public/html/)
* 选择版本**GREAT version 4.0.4**；
* Species Assembly选择**Human: GRCh37 (UCSC hg19, Feb. 2009)**；
* 将Test regions选择**BED data**，将染色体以及起始和终止的数据粘贴进去；
* 点击**Submit**，下载相应的结果。